### Sample Names
- GFP: "1_GFP-1", "2_GFP-2", "3_GFP-3"
- RBD: "4_RBD-1","5_RBD-2", "6_RBD-3"
- G1B: "7_G1B-1", "8_G1B-2", "9_G1B-3"
- G1C: "10_G1C-1", "11_G1C-2", "12_G1C-3"

In [ ]:
# Run this to allow using the %%R cell magic
%load_ext rpy2.ipython

# Suppress warnings (most notable from rpy2)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%R

# Load in all packages you will need. 
#Install with install.packages("---") if you do not have already.
library(Seurat)
library(data.table)
library(tidyverse)
library(ggplot2)
library(patchwork)
library(reshape2)
lapply(c("dplyr","Seurat","HGNChelper","openxlsx"), library, character.only = T)
source("https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/R/gene_sets_prepare.R"); 
source("https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/R/sctype_score_.R")

### Load sample and merge

In [ ]:
%%R

# Load in filtered_feature_bc_matrix directory. 
# Assign to object named counts{sample #}
counts10 <- Read10X(data.dir = "/Users/[user]/Downloads/[path]/filtered_feature_bc_matrix")

In [ ]:
%%time

# Create Seurat object. 
# Change input and project name only.
%R seurat10 <- CreateSeuratObject(counts10, project = "[1sample]", min.cells = 5, min.features = 200)

### Clustering

In [ ]:
%%time

# Normalize Data. 
# Change input name only.
%R seurat10 <- NormalizeData(seurat10, normalization.method = "LogNormalize")

In [ ]:
%%time

# Identify mitochondrial genes. 
# Change both object names only.
%R seurat10[["percent.mito"]] <- PercentageFeatureSet(object = seurat10, pattern = "mt-")

In [ ]:
%%time

# Filter based on uniqueRNA, totalRNA, and mitochondrial genes. 
# Change object name and parameters if necessary.
%R seurat10 <- subset(x = seurat10, subset = nFeature_RNA > 500 & percent.mito < 5)

In [ ]:
%%time

# Scale data. 
# Change both object names only.
%R seurat10 <- ScaleData(seurat10, vars.to.regress = c("percent.mito"))

In [ ]:
# Run to save after scaling step
%R saveRDS(seurat10, "/Users/[user]/Downloads/[path]/afterscaling.rds")

In [ ]:
%%time

# Find variable features. 
# Change only object name, parameters if necessary
%R seurat10 <- FindVariableFeatures\
(seurat10, selection.method = "mean.var.plot", \
 mean.cutoff = c(0.0125, 3), dispersion.cutoff = c(0.5, Inf))

In [ ]:
%%time

# Run PCA
%R seurat10 <- RunPCA(seurat10, seed.use = 42)

In [ ]:
%%time

# Run TSNE using parameters from paper. Change object name.
%R seurat10 <- RunTSNE(seurat10, dims.use = 1:16, do.fast = T, perplexity =  15, seed.use = 7777)

In [ ]:
%%R

# Outputs tsne plot as pdf with no clusters. 
# Check shape to make sure it looks okay
pdf("seurat_preclusterlabel.pdf", width = 10, height = 10)
plot <- DimPlot(seurat10, reduction = "tsne")  
print(plot)
dev.off()

In [ ]:
# Run this and following cell (changing save directory) to save and load seurat object as RDS. 
# Do not proceed usingvoriginal seurat object
%R saveRDS(seurat10, "/Users/dhruvkhatri/Downloads/JaesuData1/1_GDB-1/aftertsne.rds")

In [ ]:
%R seurat_copy <- readRDS("/Users/dhruvkhatri/Downloads/JaesuData1/1_GDB-1/aftertsne.rds")

In [ ]:
%%time

# Performs find neighbors for clustering later on.
%R seurat_copy <- FindNeighbors(seurat_copy, dims = 1:35)

In [ ]:
%%time

# Finds clusters based on resolution. 
# Higher resolution == more clusters. 
# Keep resolution at 5.
%R seurat_copy <- FindClusters(seurat_copy, resolution = 5, random.seed = 42)

In [ ]:
%%R
# Only run this first time
# Outputs tSNE plot as pdf. 
# Use to check if number of clusters is correct.
pdf("seurat_jaesu_data_sample10_prelabel.pdf", width = 10, height = 10)
plot <- DimPlot(seurat_copy, reduction = "tsne")  
print(plot)
dev.off()

In [ ]:
%%R

# Load in required files for SCType. 
# SCType is used for labeling clusters.

# load gene set preparation function
source("https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/R/gene_sets_prepare.R")

# load cell type annotation function
source("https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/R/sctype_score_.R")

gs_list = gene_sets_prepare\
("https://raw.githubusercontent.com/IanevskiAleksandr/sc-type/master/ScTypeDB_short.xlsx", \
 "Immune system")

In [ ]:
%%R

# Set the file path for db_ to the excel file for marker genes. 
# This file should be formatted correctly. For guidance
# on formatting, reference link in cell above in gs_list object.

# DB file
db_ = "/Users/[user]/Downloads/marker_genes.xlsx";

# Set the tissue type you are working with
# e.g. Immune system,Pancreas,Liver,Eye,Kidney,Brain,Lung,
# Adrenal,Heart,Intestine,Muscle,Placenta,Spleen,Stomach,Thymus 
tissue = "Bone marrow" 

In [ ]:
%%R

# Prepares the marker gene data for application to your results.
gs_list = gene_sets_prepare(db_, tissue)

# Performs calculation of distinguising cell types from marker genes. 
# May take a minute or two.
es.max = sctype_score(scRNAseqData = seurat_copy[["RNA"]]@scale.data, scaled = TRUE, 
                      gs = gs_list$gs_positive, gs2 = gs_list$gs_negative) 

In [ ]:
%%R
# Changes the metadata of the seurat object to include the cell types you discovered in previous steps

# NOTE: scRNAseqData parameter should correspond to your input scRNA-seq matrix. 
# In case Seurat is used, it is either pbmc[["RNA"]]@scale.data (default), 
# pbmc[["SCT"]]@scale.data, in case sctransform is used for normalization,
# or pbmc[["integrated"]]@scale.data, in case a joint analysis of multiple single-cell 
# datasets is performed.

# merge by cluster
cL_results = do.call("rbind", lapply(unique(seurat_copy@meta.data$seurat_clusters), function(cl){
  es.max.cl = sort(rowSums(es.max[ ,rownames(seurat_copy@meta.data[seurat_copy@meta.data$seurat_clusters==cl, ])])\
                   , decreasing = !0)head(data.frame(cluster = cl, type = names(es.max.cl), scores = es.max.cl, \
                ncells = sum(seurat_copy@meta.data$seurat_clusters==cl)), 10)}))

sctype_scores = cL_results %>% group_by(cluster) %>% top_n(n = 1, wt = scores)  

In [ ]:
%%R
# Performs quality check by setting clusters that were not easily disguishable as "unknown"

# set low-confident (low ScType score) clusters to "unknown"
sctype_scores$type[as.numeric(as.character(sctype_scores$scores)) < sctype_scores$ncells/4] = "Unknown"
print(sctype_scores[,1:3])

seurat_copy@meta.data$customclassif = ""
for(j in unique(sctype_scores$cluster)){
  cl_type = sctype_scores[sctype_scores$cluster==j,]; 
  seurat_copy@meta.data$customclassif[seurat_copy@meta.data$seurat_clusters == j] = as.character(cl_type$type[1])
}

In [ ]:
%%R
# Outputs final tSNE plot as pdf with clusters labeled
# Change plot titles as required

pdf("seurat_postlabeling.pdf", width = 10, height = 10)
plot <- DimPlot(seurat_copy, reduction = "tsne", label = TRUE, label.size = 4, \
        repel = TRUE, group.by = 'customclassif', ) 
print(plot+ ggtitle("[sample]"))
dev.off()

In [ ]:
# Save RDS object with clusters labeled to run DE on it later
%R saveRDS(seurat_copy, "/Users/[user]/Downloads/[path]/afterlabels.rds")

### Differential Expression Analysis & Cell Numbers

In [ ]:
# Load in RDS that has labeled clusters (afterlabels.rds)
# or just continue using seurat_copy if you just ran previous steps and skip this
%R seurat_copy <- readRDS("/Users/[user]/Downloads/[path]/afterlabels.rds")

In [ ]:
# Overwrite clusters from FindClusters with SC-Type labeled clusters in metadata
%R Idents(seurat_copy) <- "customclassif"

In [ ]:
%%R

metadata_df <- seurat_copy@meta.data

# Create a data.table from the metadata_df
md_dt <- as.data.table(metadata_df)

# Calculate the count of rows by orig.ident and customclassif
md_count <- md_dt[, .N, by = c("orig.ident", "customclassif")]

# Reshape the data
reshaped_table <- dcast(md_count, orig.ident ~ customclassif, value.var = "N")

numeric_columns <- reshaped_table %>%
  select_if(is.numeric)

# Calculate row sums for each row in reshaped_table
row_sums <- rowSums(numeric_columns <- reshaped_table %>%
  select_if(is.numeric), na.rm = TRUE)

# Add row sums as a new column to the table
reshaped_table$TotalCells <- row_sums

# Print or view the table
reshaped_table

In [ ]:
%R reshaped_table

In [ ]:
# Finds differentially expressed genes in each cell cluster. 
# Could take a while depending on how many clusters.
# Progress bar might be very messy if you run locally
%R BM.markers <- FindAllMarkers(seurat_copy, only.pos = TRUE)

In [ ]:
# If you ran previous step locally and/or it took a while to run, save markers here
%R saveRDS(BM.markers, "/Users/[user]/Downloads/[path]/afterfindmarkers.rds")

In [ ]:
%%R

# Make copy so previous step does not have to be rerun. 
# Group the genes into one object. Set n=# to how many genes
# you want per cluster on heatmap. ex: if you choose n = 5, 
# heatmap will show 5 most differentially expressed genes
# in each cluster.

BM.markers_copy <- copy(BM.markers)
BM.markers_copy <- BM.markers_copy %>% group_by(cluster) %>% top_n(n = 5, wt = avg_log2FC)

In [ ]:
# Check to make sure previous step run correctly
%R ls(BM.markers_copy)

In [ ]:
%%R

# Outputs heatmap based on previous parameters
# Set Plot Title to correct sample
pdf("sample_heatmap.pdf", width = 10, height = 10)
heatmap <- DoHeatmap(seurat_copy, features = unique(BM.markers_copy$gene), size = 2, 
          angle = 90) + scale_fill_gradientn(colors = c("blue", "white", "red")) 
print(heatmap+ ggtitle("[sample]: Differentially Expressed Genes By Cluster"))
dev.off()

### Generate Volcano Plot for potential DE genes (python)

In [ ]:
# Volcano plot is best in Python. 
# Convert dataframe to csv to import in python.
%R write.csv(BM.markers, "/Users/[user]/Downloads/[path]/diffexpressed_genes.csv")

In [ ]:
# Imports packages. 
# pip install if you do not have installed already.

import pandas as pd
import seaborn as sns
import math
import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text
import random

In [ ]:
# Load in and check data

df = pd.read_csv('/Users/[user]/Downloads/[path]/diffexpressed_genes.csv').dropna()
df.head()

In [ ]:
new_pval = df['p_val'].tolist()
for p_ind in range(len(new_pval)):
    new_pval[p_ind] = -1 * math.log(new_pval[p_ind], 10)
df_res['log10_pvalue'] = new_pval

In [ ]:
plt.figure(figsize = (6,6))

ax = sns.scatterplot(data = df, x = 'avg_log2FC', y = 'log10_pvalue',
                    hue = 'color', hue_order = ['nobody_cares', 'picked1', 'picked2', 'i_care'],
                    palette = ['lightgrey', 'orange', 'purple', 'grey'],
                    style = 'shape', style_order = ['picked3', 'picked4', 'not_important'],
                    markers = ['^', 's', 'o'], 
                    size = 'baseMean', sizes = (40, 400))

ax.axhline(2, zorder = 0, c = 'k', lw = 2, ls = '--')
ax.axvline(1, zorder = 0, c = 'k', lw = 2, ls = '--')
ax.axvline(-1, zorder = 0, c = 'k', lw = 2, ls = '--')



texts = []
for i in range(len(df)):
    if df.iloc[i].log10_pvalue > 5 and abs(df.iloc[i].avg_log2FC) > 2:
        texts.append(plt.text(x = df.iloc[i].avg_log2FC, y = df.iloc[i].log10_pvalue, s = df.iloc[i].symbol,
                             fontsize = 12, weight = 'bold'))
        
adjust_text(texts, arrowprops = dict(arrowstyle = '-', color = 'k'))

plt.legend(loc = 1, bbox_to_anchor = (1.4,1), frameon = False, prop = {'weight':'bold'})

for axis in ['bottom', 'left']:
    ax.spines[axis].set_linewidth(2)
    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.tick_params(width = 2)

plt.xticks(size = 12, weight = 'bold')
plt.yticks(size = 12, weight = 'bold')

plt.xlabel("avg_log2FC", size = 15)
plt.ylabel("$log_(10)$_pvalue", size = 15)

plt.savefig('volcanoplot.png', dpi = 300, bbox_inches = 'tight', facecolor = 'white')

plt.show()